# Mapping County Data

We are currently working on a research project that requires the analysis of data aggregated to the county level.  Spatial distributions are, as always, a concern of mine.  Since we do not yet have the data, I might as well take a bit to develop some tools that will make it easier to generate publication quality maps with [matplotlib](https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8&client=ubuntu#q=matplotlib).  The basis for this tool set will be a [Notebook](https://github.com/choct155/MiscProj/blob/master/MapLayering.ipynb) from previous work on the neighborhoods in DC.

In [1]:
#Basic Data Manipulation
import numpy as np

#Spatial I/O
import fiona

#Spatial Geometry Manipulation
from shapely.geometry import shape,LineString,Polygon,MultiPolygon
from shapely.ops import unary_union
from descartes import PolygonPatch
from rtree import index

#Visualization
from matplotlib.collections import PatchCollection
from mpltools import color

## Acquisition of County Boundaries

The first step is to capture the shapefile in question, so that we may explore the components we will leverage in generating our mapping object.  We can pull this file directly from the Census website.

In [2]:
!wget ftp://ftp2.census.gov/geo/tiger/TIGER2015/COUNTY/tl_2015_us_county.zip
!sudo unzip tl_2015_us_county.zip
!sudo mv ./tl* ./shp
!ls shp/

--2015-10-20 18:19:06--  ftp://ftp2.census.gov/geo/tiger/TIGER2015/COUNTY/tl_2015_us_county.zip
           => ‘tl_2015_us_county.zip’
Resolving ftp2.census.gov (ftp2.census.gov)... 148.129.75.35, 2610:20:2010:a09:1000:0:9481:4b23
Connecting to ftp2.census.gov (ftp2.census.gov)|148.129.75.35|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /geo/tiger/TIGER2015/COUNTY ... done.
==> SIZE tl_2015_us_county.zip ... 73995415
==> PASV ... done.    ==> RETR tl_2015_us_county.zip ... done.
Length: 73995415 (71M) (unauthoritative)

tl_2015_us_county.z 100%[=====================>]  70.57M   506KB/s   in 2m 25s 

2015-10-20 18:21:30 (500 KB/s) - ‘tl_2015_us_county.zip’ saved [73995415]

Archive:  tl_2015_us_county.zip
 extracting: tl_2015_us_county.cpg   
  inflating: tl_2015_us_county.dbf   
  inflating: tl_2015_us_county.prj   
  inflating: tl_2015_us_county.shp   
  inflating: tl_2015_us_county.shp.ea.iso.xml  
  

In [3]:
!ls shp/

tl_2015_us_county.cpg		  tl_2015_us_county.shp.iso.xml
tl_2015_us_county.dbf		  tl_2015_us_county.shp.xml
tl_2015_us_county.prj		  tl_2015_us_county.shx
tl_2015_us_county.shp		  tl_2015_us_county.zip
tl_2015_us_county.shp.ea.iso.xml
